In [75]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np


In [76]:
future = 5

def get_tecs(raw_dir = "stock_data/"):
    data_gen = DataGenerator(random=False, base_dir = raw_dir)
    data_gen.rewind()
    stateUtil = StateUtil(data_gen, future = future)
    data = []
    data_count = data_gen.max_steps()
    tec = TecAn()
    for i in tqdm(range(data_count)):
        raw = data_gen.next()
        price = raw[stateUtil.PRICE_KEY]
        amount = raw[stateUtil.AMOUNT_KEY]
        data.append([price, amount])
    
    def calcule_tecs():
        tecs = []
        print("Data {}".format(len(data)))
        df = pd.DataFrame(data, columns = ['Close', 'Volume'])
        close = df['Close']
        volume = df['Volume']
        for ta in tec.tas:
            value = ta(close, volume, 0.0, 0.0)
            tecs.append(value)
        return tecs
    
    return calcule_tecs()


In [77]:
def add_tacs(list, index, result):
    list = []
    for tec in result:
        list.append(tec.iloc[index])
    return list

In [80]:
path = "./" 

import data_util
from data_generator import DataGenerator

def load_datasets(dirs):
    sets = []  
    for raw_dir in dirs:
        data_gen = DataGenerator(random = False, base_dir = raw_dir)
        data_gen.rewind()
        full = data_gen.steps
        print("Processing {}".format(raw_dir))
        print("Recovering TECs")
        result = get_tecs(raw_dir)
        parsed = lambda list, price, amount, index: add_tacs(list, index, result)
        print("TECs recovered {}".format((result[-1].size)))
        print("Loading {} states".format(full))
        final_data = data_util.get_sets(data_gen, full, val_percentage = 0.003, path = path, use_cache=False, on_state_parsed = parsed)
        count = len(final_data[0]) + len(final_data[-1])
        print("Total to be added {}".format(count))
        sets.append(final_data)
    return sets

def conc_sets(sets):
  trainX = sets[0][0]
  trainY = sets[0][1]
  valX = sets[0][2]
  valY = sets[0][3]
  for i in range(1,  len(sets)):
    data_set = sets[i]
    trainX = np.append(data_set[0], trainX, axis = 0)
    trainY = np.append(data_set[1], trainY, axis = 0)
    valX = np.append(data_set[2], valX, axis = 0)
    valY = np.append(data_set[3], valY, axis = 0)
  return trainX, trainY, valX, valY

#sets = load_datasets(["stock_data/"])
#sets = load_datasets(["ethusdMar/", "btcusdMar/"])
sets = load_datasets(["ethusdMar/"])


trainX, trainY, valX, valY = conc_sets(sets)
np.save(path + "train" + "X.npy", trainX)
np.save(path + "train" + "Y.npy", trainY)
np.save(path + "val" + "X.npy", valX)
np.save(path + "val" + "Y.npy", valY)



Processing ethusdMar/
Recovering TECs



Data 7069
TECs recovered 7069
Loading 7069 states



Completed: (7047, 13) (7047,) (21, 13) (21,)
Total to be added 7068


In [81]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np


raw_dir = "ethusdMar/"
data_gen = DataGenerator(random=False, first_index=0, base_dir = raw_dir)
stateUtil = StateUtil(data_gen, future = 10)

In [67]:
state = data_gen.next()
print(state['timestamp'])
state = data_gen.next()
print(state['timestamp'])

1615551655
1615551655


In [68]:
state = data_gen.next()
print(state['timestamp'])
x, y, raw_state, furure_state = stateUtil.get_state(state, data_gen.index)

current = int(raw_state['timestamp'])
microcurrent = int(raw_state['microtimestamp'])

future = int(furure_state['timestamp'])
microfuture = int(furure_state['microtimestamp'])
print("{} - {} ".format(current, microcurrent))
print("{} - {} ".format(future, microfuture))
print("Diff: {}".format(future - current))
print("Y: {}".format(y))

ask = float(raw_state[stateUtil.ASKS_KEY][0][0]) 
print("Ask: {}".format(ask))
future_bid = float(furure_state[stateUtil.BIDS_KEY][0][0])
print("Bid: {}".format(future_bid))


1615551655
1615551655 - 1615551655793024 
1615551665 - 1615551665993082 
Diff: 10
Y: 0
Ask: 1773.04
Bid: 1769.4


In [9]:
result = get_tecs(raw_dir)


Data 7069


In [10]:
print("DataGenerator steps {}".format(data_gen.steps))
print("DataGenerator max_steps {}".format(data_gen.max_steps()))
print("TACS {}".format(result[0].size))


DataGenerator steps 7069
DataGenerator max_steps 7069
TACS 7069


In [11]:
data_gen = DataGenerator(random = False, base_dir = raw_dir)

In [12]:
trainY

array([0, 0, 0, ..., 0, 0, 0])